# **Práctica 4: Modelo binomial**

In [1]:
import numpy as np
from math import exp

**Introducción**

El objetivo principal de esta práctica es calcular el precio de una opción de compra (Call) y una opción de venta (Put) de estilo europeo utilizando el Modelo Binomial de Cox-Ross-Rubinstein (CRR).Este modelo es una herramienta fundamental en la valoración de derivados, ya que aproxima la evolución continua del precio de un activo subyacente mediante una sucesión de movimientos discretos de subida ($U$) o bajada ($D$). Para ello, emplearemos el concepto de valoración de riesgo neutral y la técnica de retroceso en el árbol.

**Características del Activo y la Opción**

El activo subyacente y la opción a valorar tienen las siguientes características:

In [2]:
# ============================================
# 1. PARÁMETROS DEL PROBLEMA
# ============================================
S0 = 100.0      # Precio inicial de la acción (euros)
K = 100.0       # Precio de ejercicio (Strike)
r = 0.05        # Tasa de interés anual libre de riesgo (composición continua)
u = 0.1         # Parámetro u (cambio +10%): factor de subida U = 1 + u = 1.1
d = -0.1        # Parámetro d (cambio -10%): factor de bajada D = 1 + d = 0.9
T = 1.0         # Tiempo total hasta el vencimiento (años)
N = 12          # Número de pasos o intervalos (12 meses)
tau = T / N     # Paso temporal (1/12 de año)

**Cálculo de Parámetros Fundamentales del Modelo**

Esta sección es crucial, ya que establece los valores que garantizan que el árbol se valore bajo la medida neutral al riesgo. Calculamos el factor de descuento ($R$) y la probabilidad de subida ($p$) que balancea el rendimiento esperado del activo con el rendimiento libre de riesgo

In [3]:
# ============================================
# 2. CÁLCULO DE PARÁMETROS DEL MODELO CRR
# ============================================
# Factor de crecimiento bancario por paso (R = e^(r*tau))
R = exp(r * tau)
factor_descuento = 1 / R  # Factor de descuento (R⁻¹)

# Probabilidad de Riesgo Neutro (p, probabilidad de subida)
# p = (R - D) / (U - D)
factor_subida = 1 + u
factor_bajada = 1 + d
p = (R - factor_bajada) / (factor_subida - factor_bajada)
q = 1 - p  # Probabilidad de bajada

print("=" * 50)
print("MODELO BINOMIAL DE COX-ROSS-RUBINSTEIN (CRR)")
print("=" * 50)
print(f"Precio inicial S0: {S0:.2f} €")
print(f"Número de pasos: N = {N}")
print(f"Factor de descuento (R⁻¹): {factor_descuento:.6f}")
print(f"Probabilidad de subida (p): {p:.6f}")
print(f"Probabilidad de bajada (q): {q:.6f}")

MODELO BINOMIAL DE COX-ROSS-RUBINSTEIN (CRR)
Precio inicial S0: 100.00 €
Número de pasos: N = 12
Factor de descuento (R⁻¹): 0.995842
Probabilidad de subida (p): 0.520877
Probabilidad de bajada (q): 0.479123


**Función para Construir el Árbol de Precios del Activo**

Esta función crea la matriz de precios del activo subyacente. Dado que es un árbol recombinante , el precio $S_{n, j}$ solo depende del número de subidas ($j$) y el número total de pasos ($n$), no del orden de los movimientos.

In [4]:
# ============================================
# 3. CONSTRUCCIÓN DEL ÁRBOL DE PRECIOS DEL SUBYACENTE
# ============================================
def arbol_preciosS (S0, u, d, N):
    """Crea el árbol recombinante de precios del activo subyacente (matriz S)."""
    # S[n][j] = precio en paso n con j movimientos ascendentes
    S = np.zeros((N + 1, N + 1))

    factor_subida = 1 + u
    factor_bajada = 1 + d

    for n in range(N + 1):
        for j in range(n + 1):
            # Fórmula recombinante: S_0 * U^j * D^(n-j)
            mov_subida = j
            mov_bajada = n - j
            S[n][j] = S0 * (factor_subida**mov_subida) * (factor_bajada**mov_bajada)
    return S

S_arbol = arbol_preciosS(S0, u, d, N)
print(f"\nPrecios en el primer paso:")
print(f"  S_up = {S_arbol[1, 1]:.4f} (1 subida)")
print(f"  S_down = {S_arbol[1, 0]:.4f} (0 subidas)")


Precios en el primer paso:
  S_up = 110.0000 (1 subida)
  S_down = 90.0000 (0 subidas)


**Función de Valoración de la Opción por Retroceso**

Aplicamos el método de retroceso, comenzando por calcular el payoff al vencimiento y descontando hacia atrás, nodo por nodo, el valor esperado neutral al riesgo de la opción.

In [5]:
# ============================================
# 4. VALORACIÓN DE LA OPCIÓN POR RETROCESO
# ============================================
def precio_opcion(S_tree, K, N, p, discount_factor, option_type='call'):
    """Calcula el precio de una opción europea mediante retroceso en el árbol."""
    V = np.zeros((N + 1, N + 1))
    q = 1 - p

    # 4a. Payoff al vencimiento (t = N)
    for j in range(N + 1):
        # j es el número de movimientos de subida
        if option_type == 'call':
            V[N, j] = max(S_tree[N, j] - K, 0)
        else:  # put
            V[N, j] = max(K - S_tree[N, j], 0)

    # 4b. Retroceso desde t = N-1 hasta t = 0
    for n in range(N - 1, -1, -1):
        for j in range(n + 1):
            # Fórmula de valoración neutral al riesgo: V_n = R⁻¹ * [p*V_up + q*V_down]
            V[n, j] = discount_factor * (p * V[n + 1, j + 1] + q * V[n + 1, j])

    return V

# Precio de la Call
C_arbol = precio_opcion(S_arbol, K, N, p, factor_descuento, 'call')
call_price = C_arbol[0, 0]

# Precio de la Put
P_arbol = precio_opcion(S_arbol, K, N, p, factor_descuento, 'put')
put_price = P_arbol[0, 0]

print("\n" + "=" * 50)
print("RESULTADOS DE VALUACIÓN (t=0)")
print("=" * 50)
print(f"Precio de la opción Call: {call_price:.8f} €")
print(f"Precio de la opción Put:  {put_price:.8f} €")


RESULTADOS DE VALUACIÓN (t=0)
Precio de la opción Call: 16.18683076 €
Precio de la opción Put:  11.30977321 €


**Función de Cálculo de la Cartera de Cobertura**

El Delta ($\psi$) mide la sensibilidad del precio de la opción. Su cálculo en cada nodo es el paso clave para la replicación de la opción, que es el fundamento de la valoración de riesgo neutral.

In [6]:
# ============================================
# 5. CÁLCULO DE LA COBERTURA (DELTA)
# ============================================
def calculo_delta_posicion(S, V, N, R):
    """
    Calcula los parámetros de la cartera de cobertura (Delta y Posición en Banco).
    Delta (ψ) = (V_up - V_down) / (S_up - S_down)
    """
    psi = np.zeros((N, N + 1))  # ψ[n][j]: Posición en activo (Delta)
    phi = np.zeros((N, N + 1))  # φ[n][j]: Posición en cuenta bancaria

    for n in range(N):
        for j in range(n + 1):
            # Valores de la opción en el siguiente paso
            V_up = V[n + 1, j + 1]  # Valor si sube (j+1 subidas)
            V_down = V[n + 1, j]    # Valor si baja (j subidas)

            # Precios del activo en el siguiente paso
            S_up = S[n + 1, j + 1]
            S_down = S[n + 1, j]

            # Delta (ψ)
            psi[n, j] = (V_up - V_down) / (S_up - S_down)

            # Posición en cuenta bancaria (φ).
            phi[n, j] = V[n, j] - psi[n, j] * S[n, j]

    return phi, psi

phi, psi = calculo_delta_posicion(S_arbol, C_arbol, N, R)

print("\n" + "=" * 50)
print("PARÁMETROS DE COBERTURA INICIALES")
print("=" * 50)
print(f"Delta inicial (ψ₀): {psi[0, 0]:.8f}")
print(f"Posición en banco (φ₀): {phi[0, 0]:.8f} € (Capital a invertir/pedir prestado)")


PARÁMETROS DE COBERTURA INICIALES
Delta inicial (ψ₀): 0.61976048
Posición en banco (φ₀): -45.78921704 € (Capital a invertir/pedir prestado)
